In [1]:
! pip install accelerate
! pip install transformers
! pip install datasets
! pip install bitsandbytes
! pip install peft
! pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.1 MB/s eta 0:00:00


In [1]:
from huggingface_hub import notebook_login

notebook_login()

### Load Dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("timdettmers/openassistant-guanaco")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9846
    })
    test: Dataset({
        features: ['text'],
        num_rows: 518
    })
})

In [3]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [4]:
print("Text -> ", dataset["train"]["text"][0])

Text ->  ### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power

### Tokenizer

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

### Model Bits Config

In [6]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype="float16",
                                bnb_4bit_use_double_quant=True)

### Causal LM Model

In [7]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf",
                                             quantization_config=bnb_config,
                                             device_map={"": 0})

model.config.use_cache=False
model.config.pretraining_tp=1

model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

### Lora Config

In [8]:
from peft import LoraConfig

peft_config = LoraConfig(r=64,
                         lora_alpha=32,
                         lora_dropout=0.05,
                         bias="none",
                         task_type="CAUSAL_LM")

### Trainer & Training Arguments

In [9]:
from transformers import TrainingArguments, Trainer

model_output_dir = "Llama2-timdettmers-openassistant-guanaco"

training_arguments = TrainingArguments(output_dir=model_output_dir,
                                       per_device_train_batch_size=2,
                                       per_device_eval_batch_size=2,
                                       gradient_accumulation_steps=4,
                                       optim="paged_adamw_8bit",
                                       learning_rate=2e-4,
                                       lr_scheduler_type="linear",
                                       save_strategy="epoch",
                                       evaluation_strategy='steps',
                                       eval_steps=5,
                                       logging_steps=10,
                                       num_train_epochs=5,
                                       max_steps=10,
                                       fp16=True,
                                       push_to_hub=True)

In [10]:
from trl import SFTTrainer

trainer = SFTTrainer(model=model,
                     train_dataset=train_dataset,
                     eval_dataset=test_dataset,
                     peft_config=peft_config,
                     dataset_text_field="text",
                     args=training_arguments,
                     tokenizer=tokenizer,
                     packing=False,
                     max_seq_length=512)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


### Start Training

In [11]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
5,No log,1.518359
10,1.522500,1.473583


TrainOutput(global_step=10, training_loss=1.5224509239196777, metrics={'train_runtime': 122.3559, 'train_samples_per_second': 0.654, 'train_steps_per_second': 0.082, 'total_flos': 1321910601842688.0, 'train_loss': 1.5224509239196777, 'epoch': 0.01})

### Push to hub

In [12]:
trainer.push_to_hub()

'https://huggingface.co/MohamedAhmedAE/Llama2-timdettmers-openassistant-guanaco/tree/main/'

### Inference

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("MohamedAhmedAE/Llama2-timdettmers-openassistant-guanaco", trust_remote_code=True, torch_dtype=torch.float32)


adapter_config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

In [14]:
inputs = tokenizer(
    """
    Can you write a short introduction about the relevance of the term "monopsony" in economics?
    """,
    return_tensors="pt",
    return_attention_mask=False)

In [15]:
outputs = model.generate(**inputs, max_length=512)
text = tokenizer.batch_decode(outputs)[0]
print(text)

<s> 
    Can you write a short introduction about the relevance of the term "monopsony" in economics?
    
    The term monopsony refers to a market situation in which a single buyer has the ability to set prices and control supply. This is a situation that is often seen in labor markets, where a single employer has the ability to set wages and hire workers.
    
    Monopsony can have a significant impact on the economy, as it can lead to lower wages and fewer job opportunities for workers. This can have a negative impact on the overall standard of living and can lead to economic inequality.
    
    Monopsony is an important concept in economics, as it helps to explain how markets work and how prices are determined. Understanding monopsony is essential for policymakers and economists who are trying to understand how markets work and how prices are set.
    
    Monopsony is also important for workers, as it can help them to understand how they are being paid and what their rights are